In [59]:
from google.colab import drive
# drive.mount('/content/gdrive')
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [50]:
!pip install jsonpickle

In [55]:
!pip install rake_nltk

In [51]:
nltk.download('stopwords')  
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
#Node structure for graph
class Node:

    def __init__(self,src,dest,wt):
        self.src = src
        self.dest = dest
        self.wt = wt

In [52]:
import re
import os
import glob
import json
import jsonpickle 
import math
import nltk
import pandas as pd
import pathlib
import numpy as np # linear algebra
from itertools import combinations 
from collections import defaultdict 
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from scipy import spatial
from nltk.metrics import edit_distance
from collections import defaultdict 
# from WordNet import WordNet
# from Node import Node

#Class to represent an un-directed graph using adjacency list representation 
class Graph: 
   
    def __init__(self,vertices): 
        self.V = vertices #No. of vertices 
        self.V_org = vertices 
        self.graph = defaultdict(list) # default dictionary to store graph 
        
        
    # function to add an edge to graph 
    def addEdge(self,u,v,w): 
        self.graph[u].append(Node(u,v,w))
        self.graph[v].append(Node(v,u,w))

        
    #function to print graph
    def printGraph(self):
        s = ""
        for i in self.graph:
            s = s + str(i) + " is connected to "
            print(str(i) + " is connected to ")
            for node in self.graph[i]:
                s = s + str(node.dest) + "(Weight = " + str(node.wt) + ")" + " "
                print(str(node.dest) + "(Weight = " + str(node.wt) + ")" + " ")
            s = s + "\n"
            print("\n")
        return s

    def BFS(self, s, max_levels):
        visited = set()
         
        queue = []
        wordNet = WordNet()
        queue.append((s,0,0,1))
        visited.add(s)
        level = 0
        result = {}
        while queue:
            aux = []
            result[level] = []
            
            while queue:
                s = queue.pop(0)
                visited.add(s[0])
                result[level].append(s)
                for node in self.graph[s[0]]:
                    if node.dest not in visited:
                        
#                         Wordnet Similarity
                        q1 = wordNet.clean_sentence(s[0])
                        q2 = wordNet.clean_sentence(node.dest)
                        sim = 0
                        sim = wordNet.semanticSimilarity(q1, q2)

                        sumOfCooccurence = 0
                        for chi in self.graph[node.dest]:
                            if chi.dest in visited:
                                sumOfCooccurence += chi.wt
                        aux.append((node.dest, sumOfCooccurence, level+1, sim))        
                        visited.add(node.dest)
            level += 1
            if level > max_levels:
                break
            for node in aux:
                queue.append(node)
            solution = []
            for key in result:
                for tup in result[key]:
                    if tup[2] != 0 and (tup[1] + np.exp(tup[3]))/np.exp(tup[2]) >= 2:
                        solution.append( ( tup[0], (tup[1] + np.exp(tup[3]))/np.exp(tup[2]) ) )
        return solution            
    
    def exportNetwork(self, filename = "output"):
        filename += ".json"
        obj = jsonpickle.encode(self.graph)
        with open(filename, "w") as outfile: 
            json.dump(obj, outfile)

    def importNetwork(self, filename = "output"):
        filename += ".json"
        with open(filename) as json_file:
            data = json.load(json_file)
            self.graph = jsonpickle.decode(data)
            self.V = len(self.graph)
            self.V_org = len(self.graph)

In [53]:
import pandas as pd
import pathlib

class DataHandler:
    def __init__(self, path):
        self.type = pathlib.Path(path).suffix
        self.dataset_location = path
        
        #Use the pandas dataframe to get columns 
        if self.type == '.csv':
            self.df = pd.read_csv(path)
            self.df = self.df[self.df['Domain'] == "CS "]
        elif self.type == '.xlsx':
            self.df = pd.read_excel(path)
            self.df = self.df[self.df['Domain'] == "CS "]

    def getDataframe(self):
        return self.df

In [54]:
import re
import os
import glob
import json
import jsonpickle 
import math
import nltk
import pandas as pd
import pathlib
import numpy as np # linear algebra
from itertools import combinations 
from collections import defaultdict 
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from scipy import spatial
from nltk.metrics import edit_distance
from collections import defaultdict 
# from DataHandler import DataHandler
# from Graph import Graph

class PredictPaper:
    def __init__(self, dataPath, graphPath):
        self.dh = DataHandler(dataPath)
        self.dataset = self.dh.getDataframe()
        self.dataset = self.dataset.reset_index()
        self.g = Graph(10000)
        self.g.importNetwork(graphPath)
        self.querySet = []
        self.result = {}


    def predictUtil(self, queryKeyList):
        for queryKey in queryKeyList:
            queryKey = queryKey.strip().lower()
            queryDict = self.g.BFS(queryKey, 3)
            self.querySet.extend([queryKey])
            self.querySet.extend([item[0] for item in queryDict])
        return self.querySet


    # Gives sorted dictionary result of papers and number of matching keywords 
    # between input keywords and rake extracted keywords from abstract
    def predict(self, queryKeyList, commonWordsCount):
        self.querySet = self.predictUtil(queryKeyList)
        ran = len(self.dataset['Rake keywords'])
        for i in range(1000):
            val = self.dataset['Rake keywords'][i]
            keywords = val.split(";")
            com = self.calcCommon(keywords)
            if com >= commonWordsCount:
                self.result[i] = com
        self.result = dict(sorted(self.result.items(), key = lambda kv:(kv[1], kv[0]), reverse = True))
        for key in self.result:
            self.result[key] = [self.result[key], self.dataset['area'][key].strip().lower()]
        return self.result

    def calcCommon(self, keywords):
        cou = 0
        for w1 in self.querySet:
            for w2 in keywords:
                if(w1.strip().lower() == w2.strip().lower()):
                    cou+=1
        return cou

In [56]:
from rake_nltk import Metric, Rake

class KeywordExtractor:
    def __init__(self):
        self.extractedKeywords = []

    def rakealgo(self, abstract):
        r = Rake(min_length=1, max_length=2)
        r.extract_keywords_from_text(abstract)
        result = r.get_ranked_phrases()
        topResult = result[:10]
        self.extractedKeywords.append(topResult)
        return self.extractedKeywords

In [57]:
import re
import os
import glob
import json
import jsonpickle 
import math
import nltk
import pandas as pd
import pathlib
import numpy as np # linear algebra
from itertools import combinations 
from collections import defaultdict 
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from scipy import spatial
from nltk.metrics import edit_distance
from collections import defaultdict 


class Lesk(object):

    def __init__(self, sentence):
        self.sentence = sentence
        self.meanings = {}
        for word in sentence:
            self.meanings[word] = ''

    def getSenses(self, word):
        # print word
        return wn.synsets(word.lower())

    def getGloss(self, senses):

        gloss = {}

        for sense in senses:
            gloss[sense.name()] = []

        for sense in senses:
            gloss[sense.name()] += word_tokenize(sense.definition())

        return gloss

    def getAll(self, word):
        senses = self.getSenses(word)

        if senses == []:
            return {word.lower(): senses}

        return self.getGloss(senses)

    def Score(self, set1, set2):
        # Base
        overlap = 0

        # Step
        for word in set1:
            if word in set2:
                overlap += 1

        return overlap

    def overlapScore(self, word1, word2):

        gloss_set1 = self.getAll(word1)
        if self.meanings[word2] == '':
            gloss_set2 = self.getAll(word2)
        else:
            # print 'here'
            gloss_set2 = self.getGloss([wn.synset(self.meanings[word2])])

        # print gloss_set2

        score = {}
        for i in gloss_set1.keys():
            score[i] = 0
            for j in gloss_set2.keys():
                score[i] += self.Score(gloss_set1[i], gloss_set2[j])

        bestSense = None
        max_score = 0
        for i in gloss_set1.keys():
            if score[i] > max_score:
                max_score = score[i]
                bestSense = i

        return bestSense, max_score

    def lesk(self, word, sentence):
        maxOverlap = 0
        context = sentence
        word_sense = []
        meaning = {}

        senses = self.getSenses(word)

        for sense in senses:
            meaning[sense.name()] = 0

        for word_context in context:
            if not word == word_context:
                score = self.overlapScore(word, word_context)
                if score[0] == None:
                    continue
                meaning[score[0]] += score[1]

        if senses == []:
            return word, None, None

        self.meanings[word] = max(meaning.keys(), key=lambda x: meaning[x])

        return word, self.meanings[word], wn.synset(self.meanings[word]).definition()

In [58]:
import re
import os
import glob
import json
import math
import nltk
import pandas as pd
import pathlib
import numpy as np # linear algebra
from itertools import combinations 
from collections import defaultdict 
from nltk.corpus import wordnet as wn
from nltk import word_tokenize
from scipy import spatial
from nltk.metrics import edit_distance
from collections import defaultdict 
# from Lesk import Lesk

class WordNet:
     
    def __init__(self): 
        self.STOP_WORDS = nltk.corpus.stopwords.words()
        
    def tokenize(self, q1, q2):
        return word_tokenize(q1), word_tokenize(q2)


    def posTag(self, q1, q2):
        return nltk.pos_tag(q1), nltk.pos_tag(q2)


    def stemmer(self, tag_q1, tag_q2):
        stem_q1 = []
        stem_q2 = []

        for token in tag_q1:
            stem_q1.append(stem(token))

        for token in tag_q2:
            stem_q2.append(stem(token))

        return stem_q1, stem_q2
    
    def path(self, set1, set2):
        return wn.path_similarity(set1, set2)


    def wup(self, set1, set2):
        return wn.wup_similarity(set1, set2)


    def edit(self, word1, word2):
        if float(edit_distance(word1, word2)) == 0.0:
            return 0.0
        return 1.0 / float(edit_distance(word1, word2))

    def computePath(self, q1, q2):

        R = np.zeros((len(q1), len(q2)))

        for i in range(len(q1)):
            for j in range(len(q2)):
                if q1[i][1] == None or q2[j][1] == None:
                    sim = self.edit(q1[i][0], q2[j][0])
                else:
                    sim = self.path(wn.synset(q1[i][1]), wn.synset(q2[j][1]))

                if sim == None:
                    sim = self.edit(q1[i][0], q2[j][0])

                R[i, j] = sim

        # print R

        return R

    def computeWup(self, q1, q2):
        
        R = np.zeros((len(q1), len(q2)))

        for i in range(len(q1)):
            for j in range(len(q2)):
                if q1[i][1] == None or q2[j][1] == None:
                    sim = self.edit(q1[i][0], q2[j][0])
                else:
                    sim = self.wup(wn.synset(q1[i][1]), wn.synset(q2[j][1]))

                if sim == None:
                    sim = self.edit(q1[i][0], q2[j][0])

                R[i, j] = sim

        # print R

        return R

    def overallSim(self, q1, q2, R):

        sum_X = 0.0
        sum_Y = 0.0

        for i in range(len(q1)):
            max_i = 0.0
            for j in range(len(q2)):
                if R[i, j] > max_i:
                    max_i = R[i, j]
            sum_X += max_i

        for i in range(len(q1)):
            max_j = 0.0
            for j in range(len(q2)):
                if R[i, j] > max_j:
                    max_j = R[i, j]
            sum_Y += max_j

        if (float(len(q1)) + float(len(q2))) == 0.0:
            return 0.0

        overall = (sum_X + sum_Y) / (2 * (float(len(q1)) + float(len(q2))))

        return overall

    def clean_sentence(self, val):
        "remove chars that are not letters or numbers, downcase, then remove stop words"
        regex = re.compile('([^\s\w]|_)+')
        sentence = regex.sub('', val).lower()
        sentence = sentence.split(" ")

        for word in list(sentence):
            if word in self.STOP_WORDS:
                sentence.remove(word)

        sentence = " ".join(sentence)
        return sentence
    
    def semanticSimilarity(self, q1, q2):
        tokens_q1, tokens_q2 = self.tokenize(q1, q2)
        # stem_q1, stem_q2 = stemmer(tokens_q1, tokens_q2)
        tag_q1, tag_q2 = self.posTag(tokens_q1, tokens_q2)

        sentence = []
        for i, word in enumerate(tag_q1):
            if 'NN' in word[1] or 'JJ' in word[1] or 'VB' in word[1]:
                sentence.append(word[0])

        sense1 = Lesk(sentence)
        sentence1Means = []
        for word in sentence:
            sentence1Means.append(sense1.lesk(word, sentence))

        sentence = []
        for i, word in enumerate(tag_q2):
            if 'NN' in word[1] or 'JJ' in word[1] or 'VB' in word[1]:
                sentence.append(word[0])

        sense2 = Lesk(sentence)
        sentence2Means = []
        for word in sentence:
            sentence2Means.append(sense2.lesk(word, sentence))
        # for i, word in enumerate(sentence1Means):
        #     print sentence1Means[i][0], sentence2Means[i][0]

        R1 = self.computePath(sentence1Means, sentence2Means)
        R2 = self.computeWup(sentence1Means, sentence2Means)

        R = (R1 + R2) / 2

        # print R

        return self.overallSim(sentence1Means, sentence2Means, R)

In [60]:
# from PredictPaper import PredictPaper

# Driver Function 
if __name__ == "__main__": 
    dataPath = "drive/My Drive/Colab Notebooks/Data.xlsx"
    graphPath = "drive/My Drive/Colab Notebooks/rake/output2k"
    predPap = PredictPaper(dataPath, graphPath)
    inputKey  = ['cloud computing', 'computer']
    commonWords = 3
    result = predPap.predict(inputKey, commonWords)
    print(result)

{911: [4, 'network security'], 902: [4, 'distributed computing'], 912: [3, 'distributed computing'], 909: [3, 'distributed computing'], 907: [3, 'distributed computing'], 905: [3, 'network security'], 900: [3, 'distributed computing'], 895: [3, 'distributed computing']}
